#### Activity Index

This is a Python implementation of:

Bai, J., Di, C., Xiao, L., Evenson, K. R., LaCroix, A. Z., Crainiceanu, C. M., & Buchner, D. M. (2016). An activity index for raw accelerometry data and its comparison with other activity metrics. *PloS one*, 11(8), e0160644.

##### Abstract

Accelerometers have been widely deployed in public health studies in recent years. While they collect high-resolution acceleration signals (e.g., 10–100 Hz), research has mainly focused on summarized metrics provided by accelerometers manufactures, such as the activity count (AC) by ActiGraph or Actical. Such measures do not have a publicly available formula, lack a straightforward interpretation, and can vary by software implementation or hardware type. To address these problems, we propose the physical activity index (AI), a new metric for summarizing raw tri-axial accelerometry data. We compared this metric with the AC and another recently proposed metric for raw data, Euclidean Norm Minus One (ENMO), against energy expenditure. The comparison was conducted using data from the Objective Physical Activity and Cardiovascular Health Study, in which 194 women 60–91 years performed 9 lifestyle activities in the laboratory, wearing a tri-axial accelerometer (ActiGraph GT3X+) on the hip set to 30 Hz and an Oxycon portable calorimeter, to record both tri-axial acceleration time series (converted into AI, AC, and ENMO) and oxygen uptake during each activity (converted into metabolic equivalents (METs)) at the same time. Receiver operating characteristic analyses indicated that both AI and ENMO were more sensitive to moderate and vigorous physical activities than AC, while AI was more sensitive to sedentary and light activities than ENMO. AI had the highest coefficients of determination for METs (0.72) and was a better classifier of physical activity intensity than both AC (for all intensity levels) and ENMO (for sedentary and light intensity). The proposed AI provides a novel and transparent way to summarize densely sampled raw accelerometry data, and may serve as an alternative to AC. The AI’s largely improved sensitivity on sedentary and light activities over AC and ENMO further demonstrate its advantage in studies with older adults.

##### Formula

<img src="/images/AIFormula.png" alt="Activity Index Formula" width="300">

Where $\sigma_{im}^{2}(t;H)$ denote the variance of participant $i$’s acceleration signals along axis $m$ ($m$ = 1,2,3) in the window of length $H$ starting at $t$ and $\bar{\sigma}^{2}$ is the systematic variance.

The systematic noise variance denoted by $\bar{\sigma}^{2}=\sigma_{i1}^{2}+\sigma_{i2}^{2}+\sigma_{i3}^{2}$. $\bar{\sigma}^{2}$ depends on the accuracy of the device and can be calculated using raw data in periods while the accelerometer is not moving.

#### Libs

In [1]:
import pandas as pd
from math import sqrt

#### Functions

In [2]:
def getNoiseValue(df):
    """
    Calculate the systematic noise variance based on the 'x', 'y', and 'z' columns in the provided DataFrame.

    Parameters:
    - df (pandas.DataFrame): The DataFrame containing accelerometer data.

    Returns:
    - sysnoise (float): The calculated systematic noise variance, which is the sum of variances of 'x', 'y', and 'z'.
    """
    sysnoise = df['x'].var() + df['y'].var() + df['z'].var()
    return sysnoise

In [3]:
def getActivityIndex(df, noise):
    """
    Calculate the activity index based on the variance of 'x', 'y', and 'z' columns in the provided DataFrame,
    considering a given noise level.

    Parameters:
    - df (pandas.DataFrame): The DataFrame containing data for activity analysis.
    - noise (float): The systematic noise level used for normalization.

    Returns:
    - float: The calculated activity index, which represents the normalized variability of the data.
    """
    x = (df['x'].var() - noise) / noise
    y = (df['y'].var() - noise) / noise
    z = (df['z'].var() - noise) / noise
    avg = (x+y+z)/3
    max_value = max(avg,0)
    return sqrt(max_value)

In [4]:
def getSignalActivityIndex(df, noise, freq):
    """
    Calculate activity index of a given signal over provided DataFrame, considering a given noise level.

    Parameters:
    - df (pandas.DataFrame): The DataFrame containing signal data for activity analysis.
    - noise (float): The systematic noise level used for normalization.
    - freq (int): The sampling frequency of the accelerometer.

    Returns:
    - tuple: A tuple containing the activity index fot the given signal (computed by summing up activity index of each second) and the signal duration (in seconds).
    """
    nchunks = len(df) // freq
    ai_signal = 0
    for i in range(nchunks):
        start = i*freq
        end = i*freq+freq
        ai = getActivityIndex(df.iloc[start:end], noise)
        ai_signal = ai_signal + ai
    return (ai_signal, len(df)//freq)

#### Data loading

In [5]:
sample = pd.read_csv('sample_GT3X+.csv', skiprows=15, names=['x','y','z'])

In [6]:
noise = sample[1004700:1005600]

#### Example

In [23]:
signal = sample[150:600].copy()
signal =  signal.reset_index()

In [24]:
noise = sample[1004700:1005600]

In [25]:
acc_noise = getNoiseValue(noise)

In [26]:
getSignalActivityIndex(signal, acc_noise, 30)

(454.5712212794799, 15)